In [33]:
import numpy as np
import pandas as pd
import time
import math

In [83]:
np.random.seed(2)
N_STATES=6
ACTIONS=['left','right']
EPSILON=0.9
ALPHA=.1 # Learning Rate
GAMMA=.9
MAX_EPISODES=13
FRESH_TIME=.3

In [35]:
def build_q_table(n_states,actions):
    table=pd.DataFrame(np.zeros((n_states,len(actions))),columns=actions)
    return table

In [69]:
def choose_action(state,q_table):
    state_transfer=q_table.iloc[state,:]
    if np.random.uniform()>EPSILON or state_transfer.all()==0:
        action=np.random.choice(ACTIONS)
    else:
        action=state_transfer.idxmax()
    return action

In [54]:
def update(state,action):
    if action=='left':
        next_state=max(0,state-1)
    else:
        next_state=state+1
    return next_state

In [74]:
def update_env(state,episode,steps):
    env_list=['-']*(N_STATES-1)+['T']
    if state==5:
        interaction = 'Episode %s: total_steps = %s' % (episode+1, steps)
        print('\r{}'.format(interaction))
        print('\r')
    else:
        env_list[state]='o'
        print('\r{}'.format(''.join(env_list)))

In [82]:
def reinformance_learning():
    q_table=build_q_table(N_STATES,ACTIONS)
    for i in range(MAX_EPISODES):
        state=0
        steps=0
        is_terminate=False
        update_env(state,i,steps)
        while not is_terminate:
            action=choose_action(state,q_table)
            next_state=update(state,action)
            q_pred=q_table.loc[state,action]
            if next_state==5:
                q_target=1
                is_terminate=True
            else:
                q_target=GAMMA*q_table.iloc[next_state,:].max()
            val=ALPHA*(q_target-q_pred)
            q_table.loc[state,action]+=val
#             print q_table
            state=next_state
            steps+=1
            update_env(state,i,steps)
    return q_table

In [84]:
q_table=reinformance_learning()
print(q_table)

o----T
-o---T
--o--T
-o---T
--o--T
---o-T
----oT
---o-T
----oT
---o-T
----oT
---o-T
--o--T
---o-T
--o--T
---o-T
--o--T
-o---T
o----T
-o---T
--o--T
-o---T
--o--T
-o---T
o----T
-o---T
--o--T
-o---T
o----T
o----T
o----T
o----T
-o---T
o----T
-o---T
--o--T
---o-T
----oT
Episode 1: total_steps = 38

o----T
o----T
-o---T
o----T
-o---T
o----T
-o---T
--o--T
-o---T
--o--T
-o---T
--o--T
---o-T
--o--T
-o---T
--o--T
-o---T
o----T
-o---T
--o--T
---o-T
----oT
Episode 2: total_steps = 22

o----T
-o---T
o----T
-o---T
o----T
-o---T
--o--T
---o-T
----oT
Episode 3: total_steps = 9

o----T
-o---T
--o--T
---o-T
----oT
Episode 4: total_steps = 5

o----T
-o---T
--o--T
-o---T
--o--T
---o-T
----oT
Episode 5: total_steps = 7

o----T
-o---T
--o--T
---o-T
----oT
Episode 6: total_steps = 5

o----T
-o---T
--o--T
---o-T
----oT
Episode 7: total_steps = 5

o----T
-o---T
--o--T
---o-T
----oT
Episode 8: total_steps = 5

o----T
-o---T
--o--T
---o-T
----oT
Episode 9: total_steps = 5

o----T
-o---T
--o--T
---o-T
----oT
Epis

In [72]:
def choose_action(state, q_table):
    # This is how to choose an action
    state_actions = q_table.iloc[state, :]
    if (np.random.uniform() > EPSILON) or ((state_actions == 0).all()):  # act non-greedy or state-action have no value
        action_name = np.random.choice(ACTIONS)
    else:   # act greedy
        action_name = state_actions.idxmax()    # replace argmax to idxmax as argmax means a different function in newer version of pandas
    return action_name


def get_env_feedback(S, A):
    # This is how agent will interact with the environment
    if A == 'right':    # move right
        if S == N_STATES - 2:   # terminate
            S_ = 'terminal'
            R = 1
        else:
            S_ = S + 1
            R = 0
    else:   # move left
        R = 0
        if S == 0:
            S_ = S  # reach the wall
        else:
            S_ = S - 1
    return S_, R

def update_env(S, episode, step_counter):
    # This is how environment be updated
    env_list = ['-']*(N_STATES-1) + ['T']   # '---------T' our environment
    if S == 'terminal':
        interaction = 'Episode %s: total_steps = %s' % (episode+1, step_counter)
        print('\r{}'.format(interaction))
        time.sleep(2)
        print('\r                                ')
    else:
        env_list[S] = 'o'
        interaction = ''.join(env_list)
        print('\r{}'.format(interaction))
        time.sleep(FRESH_TIME)


def rl():
    # main part of RL loop
    q_table = build_q_table(N_STATES, ACTIONS)
    for episode in range(MAX_EPISODES):
        step_counter = 0
        S = 0
        is_terminated = False
        update_env(S, episode, step_counter)
        while not is_terminated:

            A = choose_action(S, q_table)
            S_, R = get_env_feedback(S, A)  # take action & get next state and reward
            q_predict = q_table.loc[S, A]
            if S_ != 'terminal':
                q_target = R + GAMMA * q_table.iloc[S_, :].max()   # next state is not terminal
            else:
                q_target = R     # next state is terminal
                is_terminated = True    # terminate this episode

            q_table.loc[S, A] += ALPHA * (q_target - q_predict)  # update
            print q_table
            S = S_  # move to next state

            update_env(S, episode, step_counter+1)
            step_counter += 1
    return q_table


if __name__ == "__main__":
    q_table = rl()
    print('\r\nQ-table:\n')
    print(q_table)

o----T
   left  right
0   0.0    0.0
1   0.0    0.0
2   0.0    0.0
3   0.0    0.0
4   0.0    0.0
5   0.0    0.0
-o---T
   left  right
0   0.0    0.0
1   0.0    0.0
2   0.0    0.0
3   0.0    0.0
4   0.0    0.0
5   0.0    0.0
o----T
   left  right
0   0.0    0.0
1   0.0    0.0
2   0.0    0.0
3   0.0    0.0
4   0.0    0.0
5   0.0    0.0
o----T
   left  right
0   0.0    0.0
1   0.0    0.0
2   0.0    0.0
3   0.0    0.0
4   0.0    0.0
5   0.0    0.0
-o---T
   left  right
0   0.0    0.0
1   0.0    0.0
2   0.0    0.0
3   0.0    0.0
4   0.0    0.0
5   0.0    0.0
o----T
   left  right
0   0.0    0.0
1   0.0    0.0
2   0.0    0.0
3   0.0    0.0
4   0.0    0.0
5   0.0    0.0
o----T
   left  right
0   0.0    0.0
1   0.0    0.0
2   0.0    0.0
3   0.0    0.0
4   0.0    0.0
5   0.0    0.0
o----T
   left  right
0   0.0    0.0
1   0.0    0.0
2   0.0    0.0
3   0.0    0.0
4   0.0    0.0
5   0.0    0.0
o----T
   left  right
0   0.0    0.0
1   0.0    0.0
2   0.0    0.0
3   0.0    0.0
4   0.0    0.0
5   0.0

In [73]:
df = pd.DataFrame(np.ones((3,3)))

In [31]:
df.iloc[2,:].all()

True

In [34]:
np.random.choice?